# A simple function

![](img/Fibonacci_staircase_at_St_Johns_College_Cambridge_UK_Valerian_Guillot.jpg)

In [23]:
def fib(n):
    if n <= 1:
        return n
    else:
        return fib(n - 2) + fib(n - 1)

## ...can be tested interactively

In [24]:
fib(5)

5

## Looks about right, ship it.

# But it could `assert` itself...

In [25]:
def fib(n):
    """ Return the `n`th fibonacci sequence number
    >>> assert fib(5) == 5
    """
    if n <= 1:
        return n
    else:
        return fib(n - 2) + fib(n - 1)


print(fib(2))
print(fib(10))

1
55


## ... and be **unit** tested with `doctest`


In [4]:
import doctest
doctest.testmod(verbose=True)

Trying:
    assert fib(5) == 5
Expecting nothing
ok
1 items had no tests:
    __main__
1 items passed all tests:
   1 tests in __main__.fib
1 tests in 2 items.
1 passed and 0 failed.
Test passed.


TestResults(failed=0, attempted=1)

# **type** hints add intent

In [26]:
def fib_typed(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fib_typed(n - 2) + fib_typed(n - 1)

## ... but only do so much alone

In [27]:
try:
    fib_typed("foo")  # actually have to enter the code
except Exception as err:
    print(err)

'<=' not supported between instances of 'str' and 'int'


In [28]:
from beartype import beartype

@beartype
def fib_bear(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fib_bear(n - 2) + fib_bear(n - 1)

In [29]:
try:
    fib_bear("bear")  # rejects bad inputs at the door
except Exception as err:
    print(err)

@beartyped __main__.fib_bear() parameter n='bear' violates type hint <class 'int'>, as str 'bear' not instance of int.


# **typing** _does_ impact performance

In [30]:
t0 = %timeit -o fib(10)

11.8 µs ± 583 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [31]:
t1 = %timeit -o fib_typed(10)

11.8 µs ± 372 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [32]:
t2 = %timeit -o fib_bear(10)

50.1 µs ± 502 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### ... but can be used to _improve_ performance

In [33]:
%reload_ext mypyc_ipython

In [34]:
%%mypyc
def fib_c(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fib_c(n - 2) + fib_c(n - 1)

In [35]:
t3 = %timeit -o fib_c(10)

745 ns ± 1.98 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# **profile** with `snakeviz` 

In [44]:
%%file fib.py
from beartype import beartype

def fib_typed(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fib_typed(n - 2) + fib_typed(n - 1)

@beartype
def fib_bear(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fib_bear(n - 2) + fib_bear(n - 1)

Overwriting fib.py


In [45]:
%reload_ext snakeviz
%snakeviz import fib; fib.fib_typed(20); fib.fib_bear(20)

 
*** Profile stats marshalled to file '/tmp/tmpw2uif_u6'.
Embedding SnakeViz in this document...


# A larger application

In [18]:
import asyncio
from tornado import web, httpclient

In [19]:
class MainHandler(web.RequestHandler):
    async def get(self, n):
        self.write(str(fib(int(n))))

In [20]:
async def main():
    application = web.Application([
        (r"/fib/(?P<n>\d*)", MainHandler),
    ])
    application.listen(8899)
    await asyncio.Event().wait()

## And yet it runs

In [21]:
asyncio.create_task(main())

<Task pending name='Task-4' coro=<main() running at /tmp/ipykernel_27318/2913153505.py:1>>

## From a client

In [22]:
r = await httpclient.AsyncHTTPClient().fetch("http://127.0.0.1:8899/10")

HTTPClientError: HTTP 404: Not Found

In [ ]:
r.body

# **fuzz** testing with `atheris`

# **acceptance** testing with `robotframework`

# **conformance** testing with `schemathesis`

# load testing with `locust`